In [23]:
#graphlab.get_dependencies()
# DASC
#Kernel > Restart


#importamos las libreias a usar
import numpy as np
import math
import matplotlib.pyplot as plt
import graphlab
# easy_install -U scikit-fuzzy #instalar por consola
import skfuzzy as fuzz
from skfuzzy import control as ctrl
import sys

#puntos de operacion C=0.9951, T=350.4, V=13.89, FlCh=105, FlRx=82

In [3]:
#cargamos el archivo en la misma carpeta
data = graphlab.SFrame.read_csv('file.csv')

#longitud total
len (data)
data

Finished parsing file C:\Users\D A Santana\Documents\01_Univer\2018-1\03 Inteligencia\proyectofinal\red_matlab_ne\file.csv

Parsing completed. Parsed 100 lines in 0.148112 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[float,float,float,float,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file C:\Users\D A Santana\Documents\01_Univer\2018-1\03 Inteligencia\proyectofinal\red_matlab_ne\file.csv

Parsing completed. Parsed 50000 lines in 0.144493 secs.

FlujoRx,FlujoCh,Concent,Tempera,Volumen
80.16,104.75,1.0,273.15,0.1
83.468,106.81,0.99994,281.12,0.10498
83.129,104.11,0.99987,287.85,0.10998
82.207,103.81,0.99989,293.53,0.11505
84.865,104.66,0.99993,298.55,0.12019
80.703,106.06,0.99989,304.24,0.12674
81.827,106.51,0.99993,308.96,0.13308
80.74,106.56,1.0001,312.91,0.13918
83.209,105.53,1.0001,316.32,0.14539
80.743,102.52,1.0,319.48,0.15219


In [4]:
#creacion de la tabla de relacion Variables entrada Vs salida Concentracion 
entvsC = graphlab.SFrame({'Concent':data.select_column('Concent'),'FlujoCh':data.select_column('FlujoCh'),
                          'FlujoRx':data.select_column('FlujoRx')})

entvsC

Concent,FlujoCh,FlujoRx
1.0,104.75,80.16
0.99994,106.81,83.468
0.99987,104.11,83.129
0.99989,103.81,82.207
0.99993,104.66,84.865
0.99989,106.06,80.703
0.99993,106.51,81.827
1.0001,106.56,80.74
1.0001,105.53,83.209
1.0,102.52,80.743


In [5]:
#creacion de la tabla de relacion Variables entrada Vs salida Volumen 
entvsV = graphlab.SFrame({'Volumen':data.select_column('Volumen'),'FlujoCh':data.select_column('FlujoCh'),
                          'FlujoRx':data.select_column('FlujoRx')})
entvsV

FlujoCh,FlujoRx,Volumen
104.75,80.16,0.1
106.81,83.468,0.10498
104.11,83.129,0.10998
103.81,82.207,0.11505
104.66,84.865,0.12019
106.06,80.703,0.12674
106.51,81.827,0.13308
106.56,80.74,0.13918
105.53,83.209,0.14539
102.52,80.743,0.15219


In [6]:
#creacion de la tabla de relacion Variables entrada Vs salida Temp 
entvsT = graphlab.SFrame({'Tempera':data.select_column('Tempera'),'FlujoCh':data.select_column('FlujoCh'),
                          'FlujoRx':data.select_column('FlujoRx'),'TCalificacion':data.select_column('Concent')})

#Creacion tabla Clasificadora
#Temp = rango, +1
#Temp =fuera rango,-1
entvsT['TCalificacion'] = entvsT['Tempera'].apply(lambda x: +1 if x in range(345,355) else -1)

#
#entvsT['Tempera'] = entvsT['Tempera'].astype(int)
entvsT['Tempera'] = entvsT['Tempera'].astype(str)

entvsT

FlujoCh,FlujoRx,TCalificacion,Tempera
104.75,80.16,-1,273.15
106.81,83.468,-1,281.12
104.11,83.129,-1,287.85
103.81,82.207,-1,293.53
104.66,84.865,-1,298.55
106.06,80.703,-1,304.24
106.51,81.827,-1,308.96
106.56,80.74,-1,312.91
105.53,83.209,-1,316.32
102.52,80.743,-1,319.48


In [7]:
#Division en Entrenamento y Validation para cada net
training_dataC, validation_dataC = entvsC.random_split(0.8) #DB concentracion
training_dataT, validation_dataT = entvsT.random_split(0.8) #DB temperatura
training_dataV, validation_dataV = entvsV.random_split(0.8) #DB volumen

In [8]:
#Comprobacion longitud Entrenamiento 80%
len(training_dataT)
training_dataT

FlujoCh,FlujoRx,TCalificacion,Tempera
104.75,80.16,-1,273.15
106.81,83.468,-1,281.12
104.11,83.129,-1,287.85
103.81,82.207,-1,293.53
106.06,80.703,-1,304.24
106.51,81.827,-1,308.96
106.56,80.74,-1,312.91
102.52,80.743,-1,319.48
102.2,80.212,-1,324.87
102.72,83.993,-1,327.05


In [9]:
#Comprobacion longitud Validacion 20%
len(validation_dataT)
validation_dataT

FlujoCh,FlujoRx,TCalificacion,Tempera
104.66,84.865,-1,298.55
105.53,83.209,-1,316.32
107.21,80.343,-1,322.3
104.2,81.641,-1,330.8
106.99,80.224,-1,332.46
106.07,82.7,-1,337.18
102.09,84.921,-1,338.34
107.68,80.029,-1,341.94
107.34,81.704,-1,346.34
105.43,83.304,-1,346.6


In [10]:
#Aprendizaje de Flujos de Entrada al Reactor vs Temperatura de Salida
graphlab.canvas.set_target('ipynb')
training_dataT['Tempera'].show()

In [11]:
netT = graphlab.deeplearning.create(training_dataT, target ='Tempera')
print "layers Red del Flujo del Reactor vs Temperatura"
print "_______________________________________________"
print netT.layers
print "_______________________________________________"
print "Parametros de la Red"
print netT.params

layers Red del Flujo del Reactor vs Temperatura
_______________________________________________
layer[0]: FullConnectionLayer
  init_sigma = 0.01
  init_random = gaussian
  init_bias = 0
  num_hidden_units = 10
layer[1]: SigmoidLayer
layer[2]: FullConnectionLayer
  init_sigma = 0.01
  init_random = gaussian
  init_bias = 0
  num_hidden_units = 223
layer[3]: SoftmaxLayer
_______________________________________________
Parametros de la Red
{'learning_rate': 0.001, 'momentum': 0.9}


In [12]:
modelT = graphlab.neuralnet_classifier.create(training_dataT, target = 'Tempera', network = netT, validation_set = validation_dataT, 
                                         metric = ['accuracy','recall@2'])#, max_iterations = 3)

Using network:

### network layers ###
layer[0]: FullConnectionLayer
  init_sigma = 0.01
  init_random = gaussian
  init_bias = 0
  num_hidden_units = 10
layer[1]: SigmoidLayer
layer[2]: FullConnectionLayer
  init_sigma = 0.01
  init_random = gaussian
  init_bias = 0
  num_hidden_units = 223
layer[3]: SoftmaxLayer
### end network layers ###

### network parameters ###
learning_rate = 0.001
metric = accuracy,recall@2
momentum = 0.9
### end network parameters ###



Creating neuralnet using cpu

Training with batch size = 100

+-----------+----------+--------------+-------------------+---------------------+-------------------+---------------------+-----------------+

| Iteration | Examples | Elapsed Time | Training-accuracy | Validation-accuracy | Training-recall@2 | Validation-recall@2 | Examples/second |

...

| 7         | 39900    | 31.767657    | 0.014261          | 0.012960            | 0.028271          | 0.026909            | 8830.708008     |

| 8         | 39900    | 36.446872    | 0.014261          | 0.012960            | 0.028271          | 0.026909            | 8527.488281     |

| 9         | 39900    | 40.936655    | 0.014261          | 0.012960            | 0.028271          | 0.026909            | 8887.286133     |

| 10        | 39900    | 45.375074    | 0.014261          | 0.012960            | 0.028271          | 0.026909            | 8990.135742     |

+-----------+----------+--------------+-------------------+---------------------+-------------------+---------------------+-----------------+

In [13]:
#Construccion valores de prueba 
test_dataT=[]
test_dataCh=[]
test_dataRx=[]
test_dataCl=[]
from random import randrange
for k in range(1000): 
    test_dataT.append(randrange(345,355))
    test_dataCh.append(randrange(103,108))
    test_dataRx.append(randrange(79,85))
    test_dataCl.append('0')
    

test_data = graphlab.SFrame({'Tempera':test_dataT,
                             'FlujoCh':test_dataCh,
                             'FlujoRx':test_dataRx,
                             'TCalificacion':test_dataCl})

test_data['TCalificacion'] = test_data['Tempera'].apply(lambda x: +1 if x in range(345,355) else -1)

test_data['Tempera'] = test_data['Tempera'].astype(str)
test_data

FlujoCh,FlujoRx,TCalificacion,Tempera
107,82,1,351
106,81,1,353
107,83,1,352
106,80,1,346
105,83,1,351
103,84,1,348
107,84,1,349
106,81,1,348
105,81,1,346
103,84,1,350


In [14]:
#Valores a calificar en la red de entrenamiento objetivo 350, rango(345,355)=1
training_dataT['Tempera'].show

<bound method SArray.show of dtype: str
Rows: 39892
['273.15', '281.12', '287.85', '293.53', '304.24', '308.96', '312.91', '319.48', '324.87', '327.05', '329.02', '333.97', '335.71', '339.28', '340.68', '342.89', '343.67', '344.35', '344.9', '345.4', '345.8', '346.11', '346.44', '346.89', '347.2', '347.53', '347.73', '347.73', '347.84', '348.45', '348.54', '348.48', '348.55', '348.68', '348.73', '348.78', '348.82', '348.93', '349.11', '349.23', '349.31', '349.86', '349.86', '350.02', '350.24', '350.35', '350.44', '350.33', '350.32', '350.25', '350.17', '350.06', '350.25', '350.25', '350.07', '350', '350', '349.95', '349.95', '350.01', '349.96', '349.89', '350.18', '350.26', '350.43', '350.41', '350.3', '350.37', '350.26', '350.15', '350.17', '350.21', '350.16', '350.05', '350.01', '349.94', '350.02', '349.92', '349.99', '349.85', '349.74', '349.79', '349.86', '349.94', '349.91', '349.78', '349.76', '349.85', '349.96', '350.11', '350.15', '350.11', '350.14', '350.31', '350.3', '350.25',

In [15]:
#Mostramos la frecuencia de recurrencia de cada dato
training_dataT['Tempera'].show(view = 'Categorical')

In [16]:
#valores validos luego de la clasificacion
training_dataT['TCalificacion'].show(view = 'Categorical')

In [17]:
#creacion de la tabla de clasificacion
Valores_Clasificados =training_dataT[training_dataT['TCalificacion']== +1]
Valores_Rechazados   =training_dataT[training_dataT['TCalificacion']== -1]
print "Valores_Clasificados: %s" % len(Valores_Clasificados)
print "Valores_Rechazados: %s" % len(Valores_Rechazados)

Valores_Clasificados: 497
Valores_Rechazados: 39395


In [18]:
#visualizacion de la tabla
Valores_Clasificados

FlujoCh,FlujoRx,TCalificacion,Tempera
103.19,80.964,1,350
105.25,80.616,1,350
107.52,84.94,1,350
103.47,82.316,1,350
103.8,84.503,1,350
105.39,79.191,1,349
107.77,82.101,1,350
103.34,84.051,1,350
103.93,80.83,1,350
107.11,83.932,1,350


In [33]:
#ramdom de los valores clasificados
from random import choice
print(choice([Valores_Clasificados]))

# valores aleatoreos de Flujo Ch y Flujo en el reactor, para una temperatura Valida entrada fuzzy
n  =Valores_Clasificados['FlujoCh']
m  =Valores_Clasificados['FlujoRx']
ce =choice(m)
cch=choice(n)
print('-------------------------------')
print('caudal Entrada, caudal chaqueta')
print[ce,cch] #flujo Rx, #flujo Ch


+---------+---------+---------------+---------+
| FlujoCh | FlujoRx | TCalificacion | Tempera |
+---------+---------+---------------+---------+
|  103.19 |  80.964 |       1       |   350   |
|  105.25 |  80.616 |       1       |   350   |
|  107.52 |  84.94  |       1       |   350   |
|  103.47 |  82.316 |       1       |   350   |
|  103.8  |  84.503 |       1       |   350   |
|  105.39 |  79.191 |       1       |   349   |
|  107.77 |  82.101 |       1       |   350   |
|  103.34 |  84.051 |       1       |   350   |
|  103.93 |  80.83  |       1       |   350   |
|  107.11 |  83.932 |       1       |   350   |
+---------+---------+---------------+---------+
[497 rows x 4 columns]
Note: Only the head of the SFrame is printed.
You can use print_rows(num_rows=m, num_columns=n) to print more rows and columns.
-------------------------------
caudal Entrada, caudal chaqueta
[79.48, 106.82]


In [35]:
#llamamos la funcion fuzzy

# Definicion de los dominios a trabajar         
# functions
elements = sys.argv

#caudalE=85
#CaudalCh=105

#de str a numero
cch=float(cch)
ce =float(ce)


caudalE = ctrl.Antecedent(np.arange(0, 160, 1), 'caudalE')
caudalCh = ctrl.Antecedent(np.arange(0, 200, 1), 'caudalCh')
temperatura = ctrl.Consequent(np.arange(0, 700, 1), 'temperatura')
volumen = ctrl.Consequent(np.arange(0, 120, 1), 'volumen')
concentracion = ctrl.Consequent(np.arange(0, 1.5, 1), 'concentracion')

# Definicion de los conjuntos difusos 
caudalE['Bajo']  = fuzz.trapmf(caudalE.universe,[0, 0, 77, 97])
caudalE['Ideal'] = fuzz.gaussmf(caudalE.universe,5 , 82)
caudalE['Alto']  = fuzz.trapmf(caudalE.universe,[67, 87, 160, 160])

caudalCh['Bajo']  = fuzz.trapmf(caudalCh.universe,[0, 0, 100, 120])
caudalCh['Ideal'] = fuzz.gaussmf(caudalCh.universe, 5, 105)
caudalCh['Alto']  = fuzz.trapmf(caudalCh.universe,[90, 110, 200, 200])

temperatura['MuyBajo']    = fuzz.trapmf(temperatura.universe,[0, 0, 60, 120])
temperatura['MedioBajo']  = fuzz.trapmf(temperatura.universe,[30, 90, 190, 250])
temperatura['LigeroBajo'] = fuzz.trapmf(temperatura.universe,[160, 220, 320, 380])
temperatura['Ideal']      = fuzz.trimf(temperatura.universe,[300, 350, 400])
temperatura['LigeroAlto'] = fuzz.trapmf(temperatura.universe,[320, 380, 480, 540])
temperatura['MedioAlto']  = fuzz.trapmf(temperatura.universe,[450, 510, 610, 670])
temperatura['MuyAlto']    = fuzz.trapmf(temperatura.universe,[580, 640, 700, 700])

volumen['Bajo']  = fuzz.trapmf(volumen.universe,[0, 0, 10.89, 20.89])
volumen['Ideal'] = fuzz.trimf(volumen.universe,[10.89, 13.89, 16.89])
volumen['Alto']  = fuzz.trapmf(volumen.universe,[13.89, 16.89, 120, 120])

concentracion['Ideal'] = fuzz.smf(concentracion.universe,0.92, 0.9951)



#Reglas Temperatura
rule1 = ctrl.Rule(caudalCh['Bajo'] & caudalE['Bajo'], temperatura['MuyBajo'])
rule2 = ctrl.Rule(caudalCh['Ideal'] & caudalE['Bajo'] | caudalCh['Bajo'] & caudalE['Ideal'], temperatura['MedioBajo'])
rule3 = ctrl.Rule(caudalCh['Alto'] & caudalE['Bajo'], temperatura['LigeroBajo'])
rule4 = ctrl.Rule(caudalCh['Ideal'] & caudalE['Ideal'], temperatura['Ideal'])
rule5 = ctrl.Rule(caudalCh['Bajo'] & caudalE['Alto'], temperatura['LigeroAlto'])
rule6 = ctrl.Rule(caudalCh['Alto'] & caudalE['Ideal'] | caudalCh['Ideal'] & caudalE['Alto'], temperatura['MedioAlto'])
rule7 = ctrl.Rule(caudalCh['Alto'] & caudalE['Alto'], temperatura['MuyAlto'])
#Reglas Volumen
rule8 = ctrl.Rule(caudalCh['Bajo'] & caudalE['Bajo'], volumen['Bajo'])
rule9 = ctrl.Rule(caudalCh['Bajo'] & caudalE['Ideal'] | caudalCh['Ideal'] & caudalE['Bajo'] , volumen['Ideal'])
rule10 = ctrl.Rule(caudalCh['Alto'] & caudalE['Bajo'] | caudalCh['Alto'] & caudalE['Ideal'] 
| caudalCh['Alto'] & caudalE['Alto'] | caudalCh['Ideal'] & caudalE['Ideal'] 
| caudalCh['Ideal'] & caudalE['Alto'] | caudalCh['Bajo'] & caudalE['Alto'] , volumen['Alto'])

#Reglas Concentracion 
rule12 = ctrl.Rule(caudalCh['Ideal'] & caudalE['Ideal'] | caudalCh['Ideal'] & caudalE['Alto'] 
|caudalCh['Alto'] & caudalE['Ideal'] | caudalCh['Alto'] & caudalE['Alto'], concentracion['Ideal'])

#controlador Temperatura
tempe_ctrl = ctrl.ControlSystem([rule1, rule2, rule3, rule4, rule5, rule6, rule7])
tempe = ctrl.ControlSystemSimulation(tempe_ctrl)

#Controlador Volumen
vol_ctrl = ctrl.ControlSystem([rule8, rule9, rule10])
vol = ctrl.ControlSystemSimulation(vol_ctrl)

#Controlador Concentracion
conc_ctrl = ctrl.ControlSystem(rule12)
conc = ctrl.ControlSystemSimulation(conc_ctrl)

# Pass inputs to the ControlSystem using Antecedent labels with Pythonic API
# Note: if you like passing many inputs all at once, use .inputs(dict_of_data)

caudal1 = elements[1]
caudal2 = elements[2]

vol.input['caudalE']  = ce #int(caudal1)
vol.input['caudalCh'] = cch #int(caudal2)
vol.compute()

conc.input['caudalE']  = ce #int(caudal1)
conc.input['caudalCh'] = cch #int(caudal2)
conc.compute()


# Crunch the numbers
print(vol.output['volumen'])
volumen.view(sim=tempe)

print(conc.output['concentracion'])
concentracion.view(sim=tempe)
plt.show()


61.0552316342
0.638204179057
